In [2]:
import pandas as pd

df_backstreetboys = pd.read_csv('Dados\Backstreet_Boys_Lyrics_score.csv') 
df_backstreetboys = df_backstreetboys.drop(columns=["Unnamed: 0", "Unnamed: 0.1"]) # Remove colunas desnecessárias

df_arcticmonkeys = pd.read_csv('Dados\lyrics.csv')

In [3]:
# Divide o dataset em 2, um para cada banda
df_bb_treino = df_backstreetboys.iloc[:50] # Seleciona as primeiras linhas do dataset
df_bb_teste = df_backstreetboys.iloc[50:100] # Seleciona as últimas linhas do dataset

df_am_treino = df_arcticmonkeys.iloc[:50] # Seleciona as primeiras linhas do dataset
df_am_teste = df_arcticmonkeys.iloc[50:100] # Seleciona as últimas linhas do dataset

In [4]:
# Probabilidade de ser de cada banda

p_am = len(df_am_treino) / (len(df_am_treino) + len(df_bb_treino))
p_bb = len(df_bb_treino) / (len(df_am_treino) + len(df_bb_treino))

In [25]:

palavras_inuteis = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z', 'and', 'or', 'and','me','he','him','himself','she','her','herself','we','our','ourself', 'the','they','them','themselves','to', 'you','your','yourself','yourselves','it',
'its','itself','what','which','who','whom','this','that','these','those','am','is','are','was','were','been','being','have','has','had','having','do','does','did','doing','will','would','should', 'ya','can','re', 'oh','could','ought','im','youre','hes','shes','its','were','theyre','ive','youve','weve','theyve','id','youd','hed','shed','wed','theyd','ill','youll','hell','shell','well','theyll','isnt','arent','wasnt','werent','hasnt','havent','hadnt','doesnt','dont','didnt','wont','wouldnt','shant','shouldnt','cant','cannot','couldnt','mustnt','lets','thats','whos','whats','heres','theres','whens','wheres','whys','hows','a','an','the','and','but','if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after','above','below','to','from','up','upon','down','in','out','on','off','over','under','again','further','then','once','here','there','when','where','why','how','all','any','both',
'each','few','more','most','other','some','such','no','nor','not','only','own','same','so','than','too','very','can','will','just','don','should','now','ve','1','2','3','4','5','6','7','8','9','0']

def separador_palavra(text):
    return text.split() # Separa o texto em palavras a partir dos espaços

def limpar_musica(musica_am):
    musica_am = musica_am.lower()
    caracteres_a_remover = [',', '.', '?', '!', '(', ')', ';', ':', '"', "'", '-', '[', ']', '\r\n', 'verse', 'chorus', '1', '2', '&']

    for caractere in caracteres_a_remover:
        musica_am = musica_am.replace(caractere, ' ')

    return musica_am


# Probabilidade de cada palavra aparecer dado que é Arctic Monkeys
dict_am = {}
dict_palavras_am = {}

for musica_am in df_am_treino['lyrics']:
    musica_am = limpar_musica(musica_am) # Aplica a função limpar_musica a cada linha da coluna lyrics
    # dict_am['palavras'] = separador_palavra(musica_am) # Aplica a função separador_palavra a cada linha da coluna lyrics
    dict_am['palavras'] = [palavra for palavra in separador_palavra(musica_am) if palavra not in palavras_inuteis]

    df_am_novo = pd.DataFrame(dict_am)
    palavras_am = df_am_novo['palavras'].explode() # Transforma a coluna palavras em um novo df para calcular a frequência de cada palavra corretamente

    palavras_unicas_am = palavras_am.unique() # Remove palavras repetidas

    for palavra_am in palavras_unicas_am:
        if palavra_am not in dict_palavras_am:
            dict_palavras_am[palavra_am] = 1
        else:
            dict_palavras_am[palavra_am] += 1

dict_prob_am = {}

for palavra_am in dict_palavras_am:
    dict_prob_am[palavra_am] = dict_palavras_am[palavra_am] / len(df_am_treino)

dict_prob_nao_am = {}

for palavra_am in dict_palavras_am:
    dict_prob_nao_am[palavra_am] = 1 - dict_prob_am[palavra_am]

In [6]:
# Probabilidade de cada palavra aparecer em Backstreets Boys
dict_bb = {}
dict_palavras_bb = {}

for musica_bb in df_bb_treino['Lyrics']:
    musica_bb = limpar_musica(musica_bb)
    dict_bb['palavras'] = separador_palavra(musica_bb) # Aplica a função separador_palavra a cada linha da coluna lyrics
    df_bb_novo = pd.DataFrame(dict_bb)
    palavras_bb = df_bb_novo['palavras'].explode() # Transforma a coluna palavras em um novo df para calcular a frequência de cada palavra corretamente

    palavras_unicas_bb = palavras_bb.unique() # Remove palavras repetidas

    for palavra_bb in palavras_unicas_bb:
        if palavra_bb not in dict_palavras_bb:
            dict_palavras_bb[palavra_bb] = 1
        else:
            dict_palavras_bb[palavra_bb] += 1

dict_prob_bb = {}

for palavra_bb in dict_palavras_bb:
    dict_prob_bb[palavra_bb] = dict_palavras_bb[palavra_bb] / len(df_bb_treino)

dict_prob_nao_bb = {}

for palavra_bb in dict_palavras_bb:
    dict_prob_nao_bb[palavra_bb] = 1 - dict_prob_bb[palavra_bb]

In [7]:
# Probabilidade de cada palavra aparecer em ambas as bandas
dict_palavras_both = {}

for palavra_am_new in dict_palavras_am:
    dict_palavras_both[palavra_am_new] = dict_palavras_am[palavra_am_new]

for palavra_bb_new in dict_palavras_bb:
    if palavra_bb_new not in dict_palavras_both:
        dict_palavras_both[palavra_bb_new] = dict_palavras_bb[palavra_bb_new]
    else:
        dict_palavras_both[palavra_bb_new] += dict_palavras_bb[palavra_bb_new]

dict_prob_both = {}

for palavra_both in dict_palavras_both:
    dict_prob_both[palavra_both] = dict_palavras_both[palavra_both] / (len(df_am_treino) + len(df_bb_treino))

dict_prob_nao_both = {}

for palavra_both in dict_palavras_both:
    dict_prob_nao_both[palavra_both] = 1 - dict_prob_both[palavra_both]

In [8]:
# Probabilidade de ser do Arctic Monkeys dado que a palavra aparece
dict_prob_am_dado_palavra = {}

for palavra in dict_prob_am:
    dict_prob_am_dado_palavra[palavra] = (dict_prob_am[palavra] * p_am) / dict_prob_both[palavra]

# Probabilidade de ser do Backstreet Boys dado que a palavra aparece
dict_prob_bb_dado_palavra = {}

for palavra in dict_prob_bb:
    dict_prob_bb_dado_palavra[palavra] = (dict_prob_bb[palavra] * p_bb) / dict_prob_both[palavra]

In [ ]:
# Teste
def classificador(musica, dict_prob_both, dict_prob_am, dict_prob_bb):
    prob_am = 0
    prob_bb = 0
    # musica = limpar_musica(musica)

    for palavra in musica:
        if palavra not in dict_prob_both or palavra not in dict_prob_am:
            prob_am *= (dict_prob_nao_am[palavra] / dict_prob_nao_both[palavra]) * p_am
        if palavra not in dict_prob_both or palavra not in dict_prob_bb:
            prob_bb *= (dict_prob_nao_bb[palavra] / dict_prob_nao_both[palavra]) * p_bb
        if palavra in dict_prob_both or palavra in dict_prob_am:
            prob_am *= (dict_prob_am[palavra] / dict_prob_both[palavra]) * p_am
        if palavra in dict_prob_both or palavra in dict_prob_bb:
            prob_bb *= (dict_prob_bb[palavra] / dict_prob_both[palavra]) * p_bb
    
    if prob_am > prob_bb:
        return 'Arctic Monkeys'
    else: 
        return 'Backstreet Boys'
    
musica = classificador('wanna be yours', dict_prob_both, dict_prob_am, dict_prob_bb)
musica